<a href="https://colab.research.google.com/github/piyushparadkar1/piyushparadkar1/blob/main/Cohort11_M3_NB_MiniProject_02_MLR_MPI_OpenMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification Program in Computational Data Science
## A program by IISc and TalentSprint
### Mini-Project: Implementation of Multiple Linear Regression using MPI and OpenMP

## Learning Objectives

At the end of the mini-project, you will be able to :

* understand the collective communication operations like scatter, gather, broadcast 
* understand the blocking and non-blocking communication
* implement multiple linear regression and run it using MPI
* implement the multiple linear regression based predictions using OpenMP

### Dataset

The dataset chosen for this mini-project is [Combined Cycle Power Plant](https://archive.ics.uci.edu/ml/datasets/combined+cycle+power+plant). The dataset is made up of 9568 records and 5 columns. Each record contains the values for Ambient Temperature, Exhaust Vaccum, Ambient Pressure, Relative Humidity and Energy Output. 

Predicting full load electrical power output of a base load power plant is important in order to maximize the profit from the available megawatt hours.  The base load operation of a power plant is influenced by four main parameters, which are used as input variables in the dataset, such as ambient temperature, atmospheric pressure, relative humidity, and exhaust steam pressure. These parameters affect electrical power output, which is considered as the target variable.

**Note:** The data was collected over a six year period (2006-11).

## Information

#### MPI in a Nutshell

MPI stands for "Message Passing Interface". It is a library of functions (in C / Python) or subroutines (in Fortran) that you insert into source code to perform data communication between processes. MPI was developed over two years of discussions led by the MPI Forum, a group of roughly sixty people representing some forty organizations.

To know more about MPI click [here](https://hpc-tutorials.llnl.gov/mpi/)


#### Multiple Linear Regression

Multiple regression is an extension of simple linear regression. It is used when we want to predict the value of a variable based on the value of two or more other variables. The variable we want to predict is called the dependent variable (or sometimes, the outcome, target or criterion variable). The variables we are using to predict the value of the dependent variable are called the independent variables (or sometimes, the predictor, explanatory or regressor variables).

**Note:** We will be using the mpi4py Python package for MPI based code implementation

## Grading = 20 Points

**Run the below code to install mpi4py package**

In [ ]:
!pip install mpi4py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#### Importing Necessary Packages

In [ ]:
# Importing pandas
import pandas as pd 
# Importing Numpy
import numpy as np 
# Importing MPI from mpi4py package
from mpi4py import MPI 
# Importing sqrt function from the Math
from math import sqrt 
# Importing Decimal, ROUND_HALF_UP functions from the decimal package
from decimal import Decimal, ROUND_HALF_UP 
import time

#### Downloading the data

In [ ]:
#@title Download the data
!wget -qq https://cdn.iisc.talentsprint.com/CDS/Datasets/PowerPlantData.csv

### Overview

* Load the data and perform data pre-processing
* Identify the features, target and split the data into train and test
* Implement multiple Linear Regression by estimating the coefficients on the given data
* Use MPI package to distribute the data and implement `communicator`
* Define functions for each objective and make a script (.py) file to execute using MPI command
* Use OpenMP component to predict the data and calculate the error on the predicted data
* Implement the Linear Regression from `sklearn` and compare the results

#### Exercise 1: Load data (1 point)

Write a function that takes the filename as input and loads the data in a pandas dataframe with the column names as Ambient Temperature, Exhaust Vaccum, Ambient Pressure, Relative Humidity and Energy Output respectively.

**Hint:** read_csv()


In [ ]:
FILENAME = "https://cdn.iisc.talentsprint.com/CDS/Datasets/PowerPlantData.csv" # File path

# YOUR CODE HERE to Define a function to load the data

#### Exercise 2: Explore data (1 point)

Write a function that takes the data loaded using the above defined function as input and explore it.

**Hint:** You can define and check for following things in the dataset inside a function

- checking for the number of rows and columns
- summary of the dataset
- check for the null values 
- check for the duplicate values

In [ ]:
# YOUR CODE HERE  
def loadData(filename):
  data = pd.read_csv(filename)
  rows, cols = data.shape
  # print(f"dimension: rows:{rows} and cols:{cols}")
  # print(data.isnull().sum())
  # print(data.describe())
  if data.duplicated().sum() >0:
    data = data.drop_duplicates()
  data.rename(columns = {'AT':'AmbientTemperature', 'V':'ExhaustVaccum', 'AP':'AbmientPressure', 'RH':'RelativeHumidity', 'PE':'EnergyOutput'}, inplace = True)
  return data
df = loadData(FILENAME)
df.shape



(9527, 5)

In [ ]:
df.head()

,AmbientTemperature,ExhaustVaccum,AbmientPressure,RelativeHumidity,EnergyOutput
0,8.34,40.77,1010.84,90.01,480.48
1,23.64,58.49,1011.40,74.20,445.75
2,29.74,56.90,1007.15,41.91,438.76
3,19.07,49.69,1007.22,76.79,453.09
4,11.80,40.66,1017.13,97.20,464.43


#### Exercise 3: Handle missing data (1 point)

After exploring the dataset if there are any null values present in the dataset then define a function that takes data loaded using the above defined function as input and handle the null values accordingly.

**Hint:** 

- Drop the records containing the null values - dropna()
- Replace the null values with the mean/median/mode - fillna()

In [ ]:
# Function to handle missing data

# YOUR CODE HERE
def removena(data):
  for col in data:
    if data[col].isna().sum()>0:
      data[col] = data[col].filna(data[col].mean())
      
  return data

  

#### Exercise 4: Scale the data (1 point)

Write a function that takes the data after handling the missing data as input and returns the standardized data.

**Hint:**

- standardization of the data  can be performed using the below formula

$ (x - mean(x)) / std(x) $ 

In [ ]:
# Defining a function to standardize the data

# YOUR CODE HERE

def scaleData(data):
  for col in data:
    data[col] = (data[col] - data[col].mean()) / data[col].std()
  return data

  

#### Exercise 5: Feature selection (1 point)

Write a function that takes scaled data as input and returns the features and target variable values

**Hint:**

- Features: AmbientTemperature, ExhaustVaccum, AmbientPressure, RelativeHumidity 
- Target Variable: EnergyOutput

In [ ]:
# Define a function

# YOUR CODE HERE
def selectFeature(data):
  X, y = data.iloc[:,:-1], data.iloc[:, -1]
  return X, y

#### Exercise 6: Correlation (1 point)

Calculate correlation between the variables

In [ ]:
# YOUR CODE HERE 
data = loadData(FILENAME)
data = removena(data)
df = scaleData(data)
df.corr()

,AmbientTemperature,ExhaustVaccum,AbmientPressure,RelativeHumidity,EnergyOutput
AmbientTemperature,1.000000,0.843689,-0.508222,-0.543947,-0.947908
ExhaustVaccum,0.843689,1.000000,-0.415718,-0.312214,-0.869900
AbmientPressure,-0.508222,-0.415718,1.000000,0.101631,0.518687
RelativeHumidity,-0.543947,-0.312214,0.101631,1.000000,0.391175
EnergyOutput,-0.947908,-0.869900,0.518687,0.391175,1.000000


#### Exercise 7: Estimate the coefficients (2 points)

Write a function that takes features and target as input and returns the estimated coefficient values

**Hint:**

- Calculate the estimated coefficients using the below formula

$ β = (X^T X)^{-1} X^T y $ 

- transpose(), np.linalg.inv()

In [ ]:
# Calculating the coeffients

# YOUR CODE HERE
X, y = selectFeature(df)
def EstimatedCoefficent (X, y):
    X_transposed = X.transpose() # or X.transpose()
#    print ("X_transposed\n", X_transposed)
    var = (np.linalg.inv((X_transposed).dot(X)))
#    print ("\nInverse of X transpose x X\n", var)
    var2 = var.dot(X_transposed)
#    print ("\nX transpose x Inverse of X transpose x X\n", var2)
    final = var2.dot(y)
#    print ("\ny x X transpose x Inverse of X transpose x X\n\n", final)
    return final
    
# YOUR CODE HERE
coefficient = EstimatedCoefficent (X,y)
print ("Estimated Coefficient from features and target is", coefficient)

Estimated Coefficient from features and target is [-0.86225353 -0.17575667  0.02111203 -0.13486446]


#### Exercise 8: Fit the data to estimate the coefficients (2 points)

Write a function named fit which takes features and targets as input and returns the intercept and coefficient values.

**Hint:**

- create a dummy column in the features dataframe which is made up of all ones
- convert the features dataframe into numpy array
- call the estimated coefficients function which is defined above
- np.ones(), np.concatenate()

In [ ]:
df.mean()

AmbientTemperature    4.389151e-16
ExhaustVaccum        -2.535788e-16
AbmientPressure       8.927466e-15
RelativeHumidity     -8.875258e-16
EnergyOutput          9.867199e-16
dtype: float64

In [ ]:
# defining a fit function
def fit(X, y):
    # YOUR CODE HERE
    n = len(X)
    # X.loc[:,'dummy'] = np.ones(n)
    m = np.ones((n,1))
    # print(np.array(X))
    X_arr = np.concatenate((m, np.array(X)), axis=1)
    
    # print(X_arr)
    coefficient_arr = EstimatedCoefficent (X_arr,y)
    #convert df to array for calculation of mean and std deviation
    # print(coefficient_arr)
    return coefficient_arr[0], coefficient_arr[1:]
#    np.concatenate([np.ones((X_arr.shape[0], 1)), X_arr], axis=1)

intercept, coefficient_arr = fit(X, y)
print ("Coefficient", coefficient_arr)
print ("Intercept", intercept)

Coefficient [-0.86225353 -0.17575667  0.02111203 -0.13486446]
Intercept 1.0181878127082933e-15


#### Exercise 9: Predict the data on estimated coefficients (1 point)

Write a function named predict which takes features, intercept and coefficient values as input and returns the predicted values.

**Hint:**

- Fit the intercept, coefficients values in the below equation

  $y = b_0 + b_1*x + ... + b_i*x_i$

In [ ]:
 # fucntion to predict the values
def predict(X, intercept, coefficients):
    # getting length of list
    X_len = len(X)
    predictions = [0] * X_len
    # X_arr = X.to_numpy()
    for i in range(X_len):
        predictions[i] = intercept + X[i][0]*coefficients[0] + X[i][1]*coefficients[1] + X[i][2]*coefficients[2] + X[i][3]*coefficients[3]
    return predictions

y_pred = predict(np.array(X), intercept, coefficient_arr)
print ("Predicted Y", y_pred)
print ("Y", y)

Predicted Y [1.3305094084381721, -0.5344208289732253, -0.9372366116016007, 0.07631322951459175, 0.8906399947869323, 0.7720604334042337, -0.557652732956667, 0.7560295828893779, -1.2295954763674752, 1.6536523849772529, -1.1904114698805552, -0.06300460679983974, -1.2315586621260528, 1.480778336961257, 0.5959552719808257, -0.34580030115669413, 0.5593754496785835, 0.984571778936522, 1.3752126960526492, 0.01428589691420823, 0.5478978302180093, -1.5937070470758818, -0.8425454971963738, 1.5032890441528803, 0.8933765070715076, 1.7973967654459813, -0.9179988002585519, 1.3889191276216268, -0.23645166936199052, -0.8218040718644881, 0.6253299084667441, -0.2628410781673149, 1.513433941436241, -1.2297861265428105, -1.4401458691619933, 0.6370397251769362, -0.7733231215170496, -0.9047130185568818, 1.4895979207678962, -0.5199603607294254, -1.2505049954178813, -0.5066059639162414, 0.6641060452457661, -0.751340644815795, 0.13486738191818937, -0.9618990208894372, 0.44625831315091075, -0.3850460664005338, 1

#### Exercise 10: Root mean squared error (1 point)

Write a function to calculate the RMSE error. 

**Hint:**

- [How to calculate the RSME error](https://towardsdatascience.com/what-does-rmse-really-mean-806b65f2e48e)

In [ ]:
# Define a function to calculate the error

# YOUR CODE HERE
def CalcRMSE(y_pred, y):
    # getting length of list
    LENGTH = len(y_pred)
    if LENGTH != len(y):
        return -1
    return np.sqrt(((y_pred - y) ** 2).mean())

RSME = CalcRMSE(y_pred, y)
print ("RSME", RSME)

RSME 0.26752544372182074


#### Exercise 11: Split the data into train and test (1 point)

Write a function named train_test_split which takes features and targets as input and returns the train and test sets respectively.

**Hint:**

- Shuffle the data
- Consider 70 % of data as a train set and the rest of the data as a test set

In [ ]:
# YOUR CODE HERE
import math
def my_train_test_split(X, y, size):
    n_train = math.floor(size * X.shape[0])
    X_train = X[:n_train]
    y_train = y[:n_train]
    X_test = X[n_train:]
    y_test = y[n_train:]
    return X_train, y_train, X_test, y_test 

print (len(X), len(y))
X_train, y_train, X_test, y_test = my_train_test_split(X, y, .7)
print (len(X_train), len(y_train), len(X_test), len(y_test))


9527 9527
6668 6668 2859 2859


In [ ]:
X_test , y_test

(      AmbientTemperature  ExhaustVaccum  AbmientPressure  RelativeHumidity
 6684           -0.046777      -0.846063         0.069508         -2.176616
 6685            0.233971       1.626681         0.226060          0.514465
 6686            1.299202      -0.114566        -1.211523         -1.987672
 6687            1.313978       0.884148         0.266461         -1.153170
 6688            0.010985      -0.702602         0.315278          0.502142
 ...                  ...            ...              ...               ...
 9563           -0.609616      -0.423561        -0.241912         -0.027722
 9564            1.847265       1.864733        -0.494415         -0.932051
 9565           -0.492750      -0.863405         0.162093          0.364542
 9566           -0.269763       0.439575         0.899401          1.459184
 9567            0.540242      -0.235957        -0.231812         -0.143416
 
 [2859 rows x 4 columns], 6684    0.529024
 6685   -1.188204
 6686   -0.944060
 6687  

#### Exercise 12: Create a communicator (1 point)

Create a comunicator and define the rank and size

In [ ]:
#@title Default title text
# YOUR CODE HERE
from mpi4py import MPI # Importing mpi4py package from MPI module

# creating the communicator
comm = MPI.COMM_WORLD
# number of the processes running through the code i.e. rank
rank = comm.Get_rank()
# total number of processes running i.e. size
size = comm.Get_size()
root = 0
# Displaying the rank and the size of the communicator
print("rank is {} and size is {}".format(rank,size))




rank is 0 and size is 1


In [ ]:
!mpirun --allow-run-as-root -np 4 python rank.py

rank is 1 and size is 4
rank is 3 and size is 4
rank is 0 and size is 4
rank is 2 and size is 4


#### Exercise 13: Divide the data into slices (1 point)

Write a function named dividing_data which takes train features set, train target set, and size of workers as inputs and returns the sliced data for each worker.

![img](https://cdn.iisc.talentsprint.com/CDS/Images/MiniProject_MPI_DataSlice.JPG)

For Example, if there are 4 processes, slice the data into 4 equal parts with 25% ratio

**Hint:**

- Divide the Data equally among the workers
  - Create an empty list
  - Iterate over the size of workers
  - Append each slice of data to the list

In [ ]:

def dividing_data(X, y, size_of_workers):
    #Divide the data among the workers
    slice_for_each_worker = int(Decimal(X.shape[0]/size_of_workers).quantize(Decimal('1.'), rounding = ROUND_HALF_UP))      
    print('Slice of data for each worker: {}'.format(slice_for_each_worker))
    X_data_for_worker = []
    y_data_for_worker = []
    for i in range(0, size_of_workers):
        if i < size_of_workers - 1:
            X_data_for_worker.append(X[slice_for_each_worker*i:slice_for_each_worker*(i+1)])
            y_data_for_worker.append(y[slice_for_each_worker*i:slice_for_each_worker*(i+1)])
        else:
            X_data_for_worker.append(X[slice_for_each_worker*i:])
            y_data_for_worker.append(y[slice_for_each_worker*i:])
    return X_data_for_worker, y_data_for_worker


#### Exercise 14: Prepare the data in root worker to assign data for all the workers (1 point)

- When it is the root worker, perform the below operation:
    - Store the features and target values in separate variables
    - Split the data into train and test sets using the train_test_split function defined above
    - Divide the data among the workers using the dividing_data function above

In [ ]:
if rank == 0:

  X_train, y_train, X_test, y_test = my_train_test_split(X, y , .7)
  X_data_worker, y_data_worker = dividing_data(X_train, y_train, size)

Slice of data for each worker: 6668


#### Exercise 15: Scatter and gather the data (1 point)

Perform the below operations:

- Send slices of the training set(the features data X and the expected target data Y) to every worker including the root worker
    - **Hint:** scatter()
    - use `barrier()` to block workers until all workers in the group reach a Barrier, to scatter from root worker.
- Every worker should get the predicted target Y(yhat) for each slice
- Get the new coefficient of each instance in a slice
    - **Hint:** fit function defined above
- Gather the new coefficient from each worker
    - **Hint:** gather()
- Calculate the root mean square error for the test set

To know more about `scatter`, `gather` and `barrier` click [here](https://nyu-cds.github.io/python-mpi/05-collectives/)

In [ ]:
# YOUR CODE HERE
slice_X = comm.scatter(X_data_worker, root=root)
slice_y = comm.scatter(y_data_worker, root=root)

slice_X_values = slice_X.values
slice_y_values = slice_y.values




In [ ]:
intercept, coefficient_arr = fit(slice_X_values, slice_y_values)


In [ ]:
y_pred = predict(slice_X_values, intercept, coefficient_arr)
y_pred

In [ ]:
x_slice_gatherred = comm.gather(coefficient_arr, root=root)

In [ ]:
print(x_slice_gatherred[0])

[-0.85818832 -0.17681743  0.02158776 -0.13285387]


In [ ]:
y_pred = predict(np.array(X_test), intercept, x_slice_gatherred[0])
y_pred

In [ ]:
RSME = CalcRMSE(y_pred, y_test)
RSME

0.2649995162730849

#### Exercise 16: Make a script and execute everything in one place (1 point)

Write a script(.py) file which contains the code of all the above exercises in it so that you can run the code on multiple processes using MPI.

**Hint:**

- magic commands
- put MPI related code under main function
- !mpirun --allow-run-as-root -np 4 python filename.py

In [ ]:
# YOUR CODE HERE for scipt(.py) 
%%writefile linear_regf_MPI.py

# Importing pandas
import pandas as pd 
# Importing Numpy
import numpy as np 
# Importing MPI from mpi4py package
from mpi4py import MPI 
# Importing sqrt function from the Math
from math import sqrt 
# Importing Decimal, ROUND_HALF_UP functions from the decimal package
from decimal import Decimal, ROUND_HALF_UP 
import time
import math



def dividing_data(X, y, size_of_workers):
    #Divide the data among the workers
    slice_for_each_worker = int(Decimal(X.shape[0]/size_of_workers).quantize(Decimal('1.'), rounding = ROUND_HALF_UP))      
    print('Slice of data for each worker: {}'.format(slice_for_each_worker))
    X_data_for_worker = []
    y_data_for_worker = []
    for i in range(0, size_of_workers):
        if i < size_of_workers - 1:
            X_data_for_worker.append(X[slice_for_each_worker*i:slice_for_each_worker*(i+1)])
            y_data_for_worker.append(y[slice_for_each_worker*i:slice_for_each_worker*(i+1)])
        else:
            X_data_for_worker.append(X[slice_for_each_worker*i:])
            y_data_for_worker.append(y[slice_for_each_worker*i:])
    return X_data_for_worker, y_data_for_worker


def fit(X, y):
    # YOUR CODE HERE
    n = len(X)
    # X.loc[:,'dummy'] = np.ones(n)
    m = np.ones((n,1))
    # print(np.array(X))
    X_arr = np.concatenate((m, np.array(X)), axis=1)
    
    # print(X_arr)
    coefficient_arr = EstimatedCoefficent (X_arr,y)
    #convert df to array for calculation of mean and std deviation
    # print(coefficient_arr)
    return coefficient_arr[0], coefficient_arr[1:]


def my_train_test_split(X, y, size):
    n_train = math.floor(size * X.shape[0])
    X_train = X[:n_train]
    y_train = y[:n_train]
    X_test = X[n_train:]
    y_test = y[n_train:]
    return X_train, y_train, X_test, y_test 


def predict(X, intercept, coefficients):
    # getting length of list
    X_len = len(X)
    predictions = [0] * X_len
    # X_arr = X.to_numpy()
    for i in range(X_len):
        predictions[i] = intercept + X[i][0]*coefficients[0] + X[i][1]*coefficients[1] + X[i][2]*coefficients[2] + X[i][3]*coefficients[3]
    return predictions

def CalcRMSE(y_pred, y):
    # getting length of list
    LENGTH = len(y_pred)
    if LENGTH != len(y):
        return -1
    return np.sqrt(((y_pred - y) ** 2).mean())

def EstimatedCoefficent (X, y):
    X_transposed = X.transpose() # or X.transpose()
#    print ("X_transposed\n", X_transposed)
    var = (np.linalg.inv((X_transposed).dot(X)))
#    print ("\nInverse of X transpose x X\n", var)
    var2 = var.dot(X_transposed)
#    print ("\nX transpose x Inverse of X transpose x X\n", var2)
    final = var2.dot(y)
#    print ("\ny x X transpose x Inverse of X transpose x X\n\n", final)
    return final

def loadData(filename):
  data = pd.read_csv(filename)
  rows, cols = data.shape
  # print(f"dimension: rows:{rows} and cols:{cols}")
  # print(data.isnull().sum())
  # print(data.describe())
  if data.duplicated().sum() >0:
    data = data.drop_duplicates()
  data.rename(columns = {'AT':'AmbientTemperature', 'V':'ExhaustVaccum', 'AP':'AbmientPressure', 'RH':'RelativeHumidity', 'PE':'EnergyOutput'}, inplace = True)
  return data



def removena(data):
  for col in data:
    if data[col].isna().sum()>0:
      data[col] = data[col].filna(data[col].mean())
      
  return data

def scaleData(data):
  for col in data:
    data[col] = (data[col] - data[col].mean()) / data[col].std()
  return data

def selectFeature(data):
  X, y = data.iloc[:,:-1], data.iloc[:, -1]
  return X, y



def main():
  FILENAME = "https://cdn.iisc.talentsprint.com/CDS/Datasets/PowerPlantData.csv"
  df = loadData(FILENAME)
  df = removena(df)
  df = scaleData(df)
  X, y = selectFeature(df)
  # creating the communicator
  comm = MPI.COMM_WORLD
  # number of the processes running through the code i.e. rank
  rank = comm.Get_rank()
  # total number of processes running i.e. size
  size = comm.Get_size()
  root = 0
  X_data_worker = None
  y_data_worker = None
  X_test = None
  y_test = None
  if rank == root:
    X_train, y_train, X_test, y_test = my_train_test_split(X, y , .7)
    X_data_worker, y_data_worker = dividing_data(X_train, y_train, size)

  slice_X = comm.scatter(X_data_worker, root=0)
  slice_y = comm.scatter(y_data_worker, root=0)

  slice_X_values = slice_X.values
  slice_y_values = slice_y.values

  intercept, coefficient_arr = fit(slice_X_values, slice_y_values)
  y_pred = predict(slice_X_values, intercept, coefficient_arr)
  
  x_slice_gatherred = comm.gather(coefficient_arr, root=root)
  if rank == root:
    y_pred = predict(np.array(X_test), intercept, x_slice_gatherred[0])
    RSME = CalcRMSE(y_pred, y_test)
    print(f"RMSE of linear_reg using MPI:{RSME}")

main()


Overwriting linear_regf_MPI.py


In [ ]:
# YOUR CODE HERE for MPI command
!mpirun --allow-run-as-root -np 4 python linear_regf_MPI.py

Slice of data for each worker: 1667
RMSE of linear_reg using MPI:0.2658882266462222


#### Exercise 17: Implement predict using OpenMP (1 point)

Get the predictions for test data and calculate the test error(RMSE) by implementing the OpenMP (pymp)

**Hints:**

* Using the pymp.Parallel implement the predict function (use from above)

* Call the predict function by passing test data as an argument

* calculate the error (RMSE) by comparing the Actual test data and predicted test data

In [ ]:
!pip install pymp-pypi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pymp-pypi: filename=pymp_pypi-0.5.0-py3-none-any.whl size=10339 sha256=6e4d9c879e26d66967ccd5b4e172e972cd93402d5ac10643666ef5838e95acec
  Stored in directory: /root/.cache/pip/wheels/22/2a/4e/d49c406bb5eb2c04b424940de41d40b3b1481c31b9f93a13c1
Successfully built pymp-pypi


In [ ]:
import pymp
# YOUR CODE HERE


def predict(X, intercept, coefficients):
    # getting length of list
    pred_arr = pymp.shared.array(y_test.shape[0])

    with pymp.Parallel(4) as p:
      for i in p.range(len(X)):
        pred_arr[i]  = intercept + X[i][0]*coefficients[0] + X[i][1]*coefficients[1] + X[i][2]*coefficients[2] + X[i][3]*coefficients[3]
    return pred_arr



In [ ]:
y_pred_pymp = predict(np.array(X_test),intercept,coefficient_arr )

In [ ]:
rmse = CalcRMSE(y_pred_pymp, y_test)
rmse

0.2649995162730849

#### Exercise 18: Use Sklearn to compare (1 point)

Apply the Linear regression on the given data using sklearn package and compare with the above results

**Hint:**
* Split the data into train and test
* Fit the train data and predict the test data using `sklearn Linear Regression`
* Compare the coefficients and intercept with above estimated coefficients
* calculate loss (RMSE) on test data and predictions and compare

In [ ]:
# YOUR CODE HERE
#x_train.shape, y_train.shape
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
regr_linear = linear_model.LinearRegression()

regr_linear.fit(X_train, y_train)
predicted = regr_linear.predict(X_test)
mse_linear = mean_squared_error(y_test, predicted, multioutput = 'uniform_average')
print ("mse_linear rsme", np.sqrt(mse_linear))

mse_linear rsme 0.26499951627308493
